# Gravity column

This PyRST example is based on the "1ph/gravityColumn.m" example found in MRST.

## MATLAB code

```
%% My First Flow Solver: Gravity Column
% In this example, we introduce a simple pressure solver and use it to
% solve the single-phase pressure equation
%
% $$\nabla\cdot v = q, \qquad
%    v=\textbf{--}\frac{K}{\mu} \bigl[\nabla p+\rho g\nabla z\bigr],$$
%
% within the domain [0,1]x[0,1]x[0,30] using a Cartesian grid with
% homogeneous isotropic permeability of 100 mD. The fluid has density 1000
% kg/m^3 and viscosity 1 cP and the pressure is 100 bar at the top of the
% structure.
%
% The purpose of the example is to show the basic steps for setting up,
% solving, and visualizing a flow problem. More details on the grid
% structure, the structure used to hold the solutions, and so on, are given
% in the <simpleBC.html basic flow-solver tutorial>.
try
    require incomp
catch %#ok<CTCH>
    mrstModule add incomp
end

%% Define the model
% To set up a model, we need: a grid, rock properties (permeability), a
% fluid object with density and viscosity, and boundary conditions.
gravity reset on
G          = cartGrid([1, 1, 30], [1, 1, 30]);
G          = computeGeometry(G);
rock.perm  = repmat(0.1*darcy(), [G.cells.num, 1]);
fluid      = initSingleFluid('mu' ,    1*centi*poise, ...
                             'rho', 1014*kilogram/meter^3);
bc  = pside([], G, 'TOP', 100.*barsa());

%% Assemble and solve the linear system
% To solve the flow problem, we use the standard two-point
% flux-approximation method (TPFA), which for a Cartesian grid is the same
% as a classical seven-point finite-difference scheme for Poisson's
% equation. This is done in two steps: first we compute the
% transmissibilities and then we assemble and solve the corresponding
% discrete system.
T   = computeTrans(G, rock);
sol = incompTPFA(initResSol(G, 0.0), G, T, fluid, 'bc', bc);

%% Plot the face pressures
clf
plotFaces(G, 1:G.faces.num, convertTo(sol.facePressure, barsa()));
set(gca, 'ZDir', 'reverse'), title('Pressure [bar]')
view(3), colorbar
set(gca,'DataAspect',[1 1 10]);
%%
displayEndOfDemoMessage(mfilename)
```

The rest of the example will walk through this example line for line, displaying the PyRST examples.

## Setup

In [1]:
# For Python 3 compatibility
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np

try:
    import pyrst
except ImportError:
    # Add the PyRST path to Python path.
    import sys
    import os
    # Directory where the pyrst module is found
    pyrst_import_path = os.path.normpath(os.path.join(os.getcwd(), "..", "..", ".."))
    sys.path.append(pyrst_import_path)
    import pyrst

import pyrst.incomp
import pyrst.gridprocessing
import pyrst.utils

In [2]:
help(pyrst)

Help on package pyrst:

NAME
    pyrst - Python Reservoir Simulation Toolbox.

FILE
    /home/shomea/a/andreros/pyrst/pyrst/__init__.py

PACKAGE CONTENTS
    gridprocessing
    incomp (package)
    io
    utils (package)

DATA
    __all__ = ['gridprocessing', 'io', 'incomp']




## Import the "incomp" module
```
try
    require incomp
catch %#ok<CTCH>
    mrstModule add incomp
end
```

In [3]:
#import pyrst.incomp...

```
%% Define the model
% To set up a model, we need: a grid, rock properties (permeability), a
% fluid object with density and viscosity, and boundary conditions.
gravity reset on
```

In [4]:
# PyRST uses a single module-level variable to set the gravity
pyrst.gravity = np.array([0, 0, 9.80665])

## Build the grid, setup fluid

```G          = cartGrid([1, 1, 30], [1, 1, 30]);
G          = computeGeometry(G);
rock.perm  = repmat(0.1*darcy(), [G.cells.num, 1]);
fluid      = initSingleFluid('mu' ,    1*centi*poise, ...
                             'rho', 1014*kilogram/meter^3);
                             ```

In [5]:
G = pyrst.gridprocessing.cartGrid([1, 1, 30], [1, 1, 30])
print(G)

<PyRST grid
  gridType: [u'tensorGrid', u'cartGrid']
  cells: ['facePos', 'num', 'indexMap', 'faces']
  cartDims: [ 1  1 30]
  gridDim: 3
  faces: ['neighbors', 'nodes', 'num', 'tag', 'nodePos']
  nodes: ['num', 'coords']
>


In [6]:
# Computing the geometry modifies the original grid in place, no assignment is necessary.
# Note that specifying the module is necessary in PyRST.
# This avoids polluting the global namespace, at the cost of more verbosity.
pyrst.gridprocessing.computeGeometry(G)

INFO:pyrst:Computing normals, areas and centroids...
INFO:pyrst:Computing cell volumes and centroids


In [7]:
print(G)

<PyRST grid
  gridType: [u'tensorGrid', u'cartGrid', u'computeGeometry']
  cells: ['facePos', 'centroids', 'num', 'volumes', 'faces', 'indexMap']
  cartDims: [ 1  1 30]
  gridDim: 3
  faces: ['neighbors', 'nodePos', 'tag', 'centroids', 'num', 'normals', 'nodes', 'areas']
  nodes: ['num', 'coords']
>


In [ ]:
rock